In [ ]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    #options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [ ]:
driver = setWebdriver()

query = input("검색어를 입력하세요: ") # 검색어 입력
start_date = re.sub(r'[^0-9]', '', input("시작 날짜를 yyyy.mm.dd 형식으로 입력하세요: ")) # 시작 날짜(정규식을 이용하여 숫자만 추출)
end_date = re.sub(r'[^0-9]', '', input("종료 날짜를 yyyy.mm.dd 형식으로 입력하세요: "))    # 종료 날짜(정규식을 이용하여 숫자만 추출)

url = f'https://search.naver.com/search.naver?ssc=tab.blog.all&query={query}&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom{start_date}to{end_date}'
driver.get(url)

tit_list = []
url_list = []
img_list = []

# 블로그 글 element 수집
articles = driver.find_elements(By.CLASS_NAME, "detail_box")
print(articles)

for article in tqdm(articles, desc="articles..."):    
    try:
        # title
        title = article.find_element(By.CLASS_NAME, "title_link").text;      tit_list.append(title);       print(f'Title: {title}')
        # url
        url = article.find_element(By.CLASS_NAME, "title_link");
        url = url.get_attribute("href")
        url_list.append(url);           print(f'URL: {url}')
        # image
        imgSrc_list = []
        try:   
            img_boxes = article.find_elements(By.CLASS_NAME, "thumb_item");         #print(f'img_boxes: {img_boxes}')
            if (img_boxes): # 여러개일 경우
                for img_box in img_boxes:
                    img_element = img_box.find_element(By.TAG_NAME, "img")
                    img_src = img_element.get_attribute("src");                      
                    imgSrc_list.append(img_src);    #print(f'img_src: {img_src}')
                img_list.append(imgSrc_list)
            else:   # 한 개일 경우
                img = article.find_element(By.CLASS_NAME, "img")
                img_src = img.get_attribute("src");
                img_list.append(img_src)
        except: # 이미지가 없을 경우
            img_list.append(None)
        print(f'img_src: {img_list[-1]}')
    except:
        break

    print('-'*200)
    time.sleep(rd.uniform(1.5, 2.5))

df = pd.DataFrame({
    "title": tit_list,
    "url": url_list,
    "images": img_list
})
print(df)

[<selenium.webdriver.remote.webelement.WebElement (session="f815aa297eff5208f0f79a391f2e7dfd", element="f.F7F836E4C50AA3001101D3DC135C2B08.d.B0E0221CCD1EB2D20196E463BBFEBE39.e.26")>, <selenium.webdriver.remote.webelement.WebElement (session="f815aa297eff5208f0f79a391f2e7dfd", element="f.F7F836E4C50AA3001101D3DC135C2B08.d.B0E0221CCD1EB2D20196E463BBFEBE39.e.27")>, <selenium.webdriver.remote.webelement.WebElement (session="f815aa297eff5208f0f79a391f2e7dfd", element="f.F7F836E4C50AA3001101D3DC135C2B08.d.B0E0221CCD1EB2D20196E463BBFEBE39.e.28")>, <selenium.webdriver.remote.webelement.WebElement (session="f815aa297eff5208f0f79a391f2e7dfd", element="f.F7F836E4C50AA3001101D3DC135C2B08.d.B0E0221CCD1EB2D20196E463BBFEBE39.e.29")>, <selenium.webdriver.remote.webelement.WebElement (session="f815aa297eff5208f0f79a391f2e7dfd", element="f.F7F836E4C50AA3001101D3DC135C2B08.d.B0E0221CCD1EB2D20196E463BBFEBE39.e.30")>, <selenium.webdriver.remote.webelement.WebElement (session="f815aa297eff5208f0f79a391f2e7d

articles...:   0%|          | 0/30 [00:00<?, ?it/s]

Title: AI투자 핀트 미국주식 소액투자 하는법 및 이벤트 소식
URL: https://m.blog.naver.com/kimgr1010/223394424045
img_src: ['https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMThfMjEg%2FMDAxNzEwNzY2ODM5OTA5.J4-qCsVGbo3Ay6RPxNLmIYfdT0jEXuV4NjKvSX9SDwsg.cX6B6Ic3CxxNrYmQB5vfdOnbiXx9c4ngCIfzt8xEQJQg.PNG%2F%25C1%25A6%25B8%25F1%25C0%25BB-%25C0%25D4%25B7%25C2%25C7%25D8%25C1%25D6%25BC%25BC%25BF%25E4_-001.png%23400x400&type=f238_208', 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMThfMjY2%2FMDAxNzEwNzY3NTc0Njgz.Fw2Gft3EYoPq9grRGbZ1PSkfc8bwNGThei1SewIYR6Yg.vHxz_wpND-Vw7k_OA0GSyPuEUEp3L5ydDmF3AJ8shOwg.PNG%2F%25C1%25A6%25B8%25F1%25C0%25BB-%25C0%25D4%25B7%25C2%25C7%25D8%25C1%25D6%25BC%25BC%25BF%25E4_-002_%25281%2529.png%231080x640&type=f238_208', 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMThfMjk3%2FMDAxNzEwNzY3NTc0MjA3.Yf9dSByS0hdfV74JiZGApzHEGrrmQJblRif7DUdkfVUg.nTw0tRpoPsHdW0FCUv1bWDbhJTYfMIaIL4MFl2PgNd4g.PNG%2

articles...:   3%|▎         | 1/30 [00:02<01:21,  2.81s/it]

Title: 3월 3주차 ) FOMC 결과로 빅테크 상승, 달러강세전환과 엔화약세, 삼성전자 8만 가나요?(+ AI 투자기록)
URL: https://m.blog.naver.com/mayo51/223393808284
img_src: ['https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMjRfMjkx%2FMDAxNzExMjgwMzg3Mzkz.A4ME8I1z0If0ch5yotBEGt9YqeNaPzMU21743pvL6Pkg.E_d2h9virn3tD_Ye4JeCQ9TorQ7Y-cq_YGfq1m9C260g.PNG%2F%25C1%25A6%25B8%25F1%25C0%25BB_%25C0%25D4%25B7%25C2%25C7%25D8%25C1%25D6%25BC%25BC%25BF%25E4_-011_%252854%2529.png%231080x1080&type=f238_208', 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMjRfMjI0%2FMDAxNzExMjc0NDEyODgy.GWnOWYULLemxfUinPMEgbTyqpSaAPB2GJQAR8OKgNksg.WTpUjCuETm5QGx-pH926ikln8FgAiAAfWATqNdbXh6cg.PNG%2F%25C1%25A6%25B8%25F1%25C0%25BB_%25C0%25D4%25B7%25C2%25C7%25D8%25C1%25D6%25BC%25BC%25BF%25E4____%25BA%25B9%25BB%25E7%25BA%25BB_%25BA%25B9%25BB%25E7%25BA%25BB-001_%25282%2529.png%231355x806&type=f238_208', 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMjRfMjcx%2FMDAxNzE

...

Title: 영어 중국어 공부 AI 리딩톡 하나면 영어사전 번역기 준비 끝
URL: https://m.blog.naver.com/dbswndus84/223393912123
img_src: https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMjRfMTMw%2FMDAxNzExMjcxNTU2ODI2.Q8LwMgy7FzH6hDuk6Bx6HqiXOi52CWo_TLFaD2xMcGAg.hxjPcctSEN2XQmjRZN48anDdIM88dTGtEczSvEXwQN4g.PNG%2F%25C1%25A6%25B8%25F1%25C0%25BB-%25C0%25D4%25B7%25C2%25C7%25D8%25C1%25D6%25BC%25BC%25BF%25E4_-001.png%231080x1080&type=ff192_192
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


articles...:  97%|█████████▋| 29/30 [01:04<00:02,  2.09s/it]

Title: 오늘 이슈 <AI 서비스 발전으로 SaaS와 클라우드 시장이 급 팽장 중이다 >
URL: https://m.blog.naver.com/jangannet/223393402739
img_src: https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMjRfMjAg%2FMDAxNzExMjQxODU1NDA1.WSgUPdhNOA2VtW3X_Cw12STZt2bRSkrdOoyQ4OtECgQg.IeR4zaC8uKs6B4_GVVHvYnTqtzAYY4tXHOjQNf2Z9GAg.PNG%2Fimg-0wPZEDefu0pQ8t3jVwBYQM7Z.png%231024x1024&type=ff192_192
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


articles...: 100%|██████████| 30/30 [01:06<00:00,  2.23s/it]

                                                title  \
0                      AI투자 핀트 미국주식 소액투자 하는법 및 이벤트 소식   
1   3월 3주차 ) FOMC 결과로 빅테크 상승, 달러강세전환과 엔화약세, 삼성전자 8...   
2                    미국주식 한국주식 AI 간편 투자 플랫폼, 핀트에서 해볼까   
3                  포토샵 AI 생성형 자동 채우기 방법 | 선택영역 배경 늘리기   
4                  AI프로필 제작 캐럿으로 나만의 인공지능 사진 및 그림 만들기   
5         투자사기 공범으로 체포, 경찰조사를 앞두고 있다면 - AI 투자, 리딩방 사기   
6                       삼성 비스포크 AI 콤보 에어살균/탈취 기능 사용후기   
7                엔비디아 GTC 2024 통해 보는 주가 전망, AI칩 로봇 공개   
8                   유튜브뮤직 구글홈 미니 네스트 AI 스피커 연결 후 음악듣기   
9                          아이폰16 출시일 디자인 생성형 AI 탑재되나?   
10                            갤럭시 S24 가격 스펙 AI 기능 총정리   
11                   렌더토큰 시세 전망, 10x 갈 수 있는 AI 진짜 대장.   
12                  화상 영어교육 서비스는 살아남을 수 있을까? (사람과 AI)   
13                아이폰16 출시일 생성형 AI기능 적용되고 울트라 모델은 없을듯   
14  스픽새로 발견한 AI 프리톡 기능 | 프리톡 쉽게 하는 순서| 스픽 가격 | 스픽 ...   
15   AI 기능을 강화하고 차별화된 편의 기능을 대거 탑재한 ‘비스포크 냉장고’ 신제품 출시   
16      “AI로 차별화된 쾌적한 바람” 간편하게 